In [20]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms

In [21]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [22]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 20, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [24]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.177
[1,  4000] loss: 1.801
[1,  6000] loss: 1.608
[1,  8000] loss: 1.516
[1, 10000] loss: 1.443
[1, 12000] loss: 1.420
[2,  2000] loss: 1.335
[2,  4000] loss: 1.306
[2,  6000] loss: 1.266
[2,  8000] loss: 1.233
[2, 10000] loss: 1.222
[2, 12000] loss: 1.190
[3,  2000] loss: 1.104
[3,  4000] loss: 1.109
[3,  6000] loss: 1.084
[3,  8000] loss: 1.072
[3, 10000] loss: 1.074
[3, 12000] loss: 1.066
[4,  2000] loss: 0.979
[4,  4000] loss: 1.005
[4,  6000] loss: 0.986
[4,  8000] loss: 0.953
[4, 10000] loss: 0.971
[4, 12000] loss: 0.995
[5,  2000] loss: 0.884
[5,  4000] loss: 0.898
[5,  6000] loss: 0.904
[5,  8000] loss: 0.914
[5, 10000] loss: 0.911
[5, 12000] loss: 0.907
[6,  2000] loss: 0.829
[6,  4000] loss: 0.846
[6,  6000] loss: 0.860
[6,  8000] loss: 0.867
[6, 10000] loss: 0.863
[6, 12000] loss: 0.863
[7,  2000] loss: 0.779
[7,  4000] loss: 0.778
[7,  6000] loss: 0.809
[7,  8000] loss: 0.821
[7, 10000] loss: 0.830
[7, 12000] loss: 0.807
[8,  2000] loss: 0.725
[8,  4000] 

In [25]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        images = images.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 63 %


In [26]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        images = images.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 44, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(44, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


# since we're not training, we don't need to calculate the gradients for our outputs
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        images =images.cuda()
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

[1,  2000] loss: 2.305
[1,  4000] loss: 2.305
[1,  6000] loss: 2.305
[1,  8000] loss: 2.303
[1, 10000] loss: 2.305
[1, 12000] loss: 2.304
[2,  2000] loss: 2.305
[2,  4000] loss: 2.305
[2,  6000] loss: 2.305
[2,  8000] loss: 2.305
[2, 10000] loss: 2.304
[2, 12000] loss: 2.304
[3,  2000] loss: 2.305
[3,  4000] loss: 2.305
[3,  6000] loss: 2.304
[3,  8000] loss: 2.304
[3, 10000] loss: 2.305
[3, 12000] loss: 2.305
[4,  2000] loss: 2.305
[4,  4000] loss: 2.305
[4,  6000] loss: 2.305
[4,  8000] loss: 2.305
[4, 10000] loss: 2.304
[4, 12000] loss: 2.305
[5,  2000] loss: 2.304
[5,  4000] loss: 2.305
[5,  6000] loss: 2.305
[5,  8000] loss: 2.304
[5, 10000] loss: 2.305
[5, 12000] loss: 2.305
Finished Training
Accuracy of the network on the 10000 test images: 10 %


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 56, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(56, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


# since we're not training, we don't need to calculate the gradients for our outputs
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        images =images.cuda()
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

[1,  2000] loss: 2.305
[1,  4000] loss: 2.305
[1,  6000] loss: 2.306
[1,  8000] loss: 2.303
[1, 10000] loss: 2.305
[1, 12000] loss: 2.305
[2,  2000] loss: 2.305
[2,  4000] loss: 2.304
[2,  6000] loss: 2.304
[2,  8000] loss: 2.305
[2, 10000] loss: 2.305
[2, 12000] loss: 2.304
[3,  2000] loss: 2.305
[3,  4000] loss: 2.304
[3,  6000] loss: 2.303
[3,  8000] loss: 2.305
[3, 10000] loss: 2.306
[3, 12000] loss: 2.304
[4,  2000] loss: 2.305
[4,  4000] loss: 2.304
[4,  6000] loss: 2.304
[4,  8000] loss: 2.304
[4, 10000] loss: 2.305
[4, 12000] loss: 2.305
[5,  2000] loss: 2.303
[5,  4000] loss: 2.305
[5,  6000] loss: 2.305
[5,  8000] loss: 2.306
[5, 10000] loss: 2.304
[5, 12000] loss: 2.305
Finished Training
Accuracy of the network on the 10000 test images: 10 %


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=56, kernel_size=3, padding=1),
            nn.BatchNorm2d(56),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=56, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data                       # this is what you had
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')



[1,  2000] loss: 1.868
[1,  4000] loss: 1.489
[1,  6000] loss: 1.309
[1,  8000] loss: 1.132
[1, 10000] loss: 1.019
[1, 12000] loss: 0.958
[2,  2000] loss: 0.834
[2,  4000] loss: 0.824
[2,  6000] loss: 0.794
[2,  8000] loss: 0.753
[2, 10000] loss: 0.729
[2, 12000] loss: 0.730
[3,  2000] loss: 0.584
[3,  4000] loss: 0.590
[3,  6000] loss: 0.601
[3,  8000] loss: 0.592
[3, 10000] loss: 0.569
[3, 12000] loss: 0.559
[4,  2000] loss: 0.432
[4,  4000] loss: 0.459
[4,  6000] loss: 0.469
[4,  8000] loss: 0.455
[4, 10000] loss: 0.440
[4, 12000] loss: 0.459
[5,  2000] loss: 0.318
[5,  4000] loss: 0.333
[5,  6000] loss: 0.344
[5,  8000] loss: 0.365
[5, 10000] loss: 0.353
[5, 12000] loss: 0.366
Finished Training


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        images =images.cuda()
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 81 %


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=20, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data                       # this is what you had
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')



[1,  2000] loss: 1.875
[1,  4000] loss: 1.486
[1,  6000] loss: 1.274
[1,  8000] loss: 1.108
[1, 10000] loss: 1.042
[1, 12000] loss: 0.970
[2,  2000] loss: 0.831
[2,  4000] loss: 0.833
[2,  6000] loss: 0.796
[2,  8000] loss: 0.774
[2, 10000] loss: 0.745
[2, 12000] loss: 0.734
[3,  2000] loss: 0.600
[3,  4000] loss: 0.590
[3,  6000] loss: 0.606
[3,  8000] loss: 0.578
[3, 10000] loss: 0.577
[3, 12000] loss: 0.595
[4,  2000] loss: 0.452
[4,  4000] loss: 0.456
[4,  6000] loss: 0.466
[4,  8000] loss: 0.452
[4, 10000] loss: 0.467
[4, 12000] loss: 0.449
[5,  2000] loss: 0.322
[5,  4000] loss: 0.331
[5,  6000] loss: 0.367
[5,  8000] loss: 0.371
[5, 10000] loss: 0.386
[5, 12000] loss: 0.372
Finished Training


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        images =images.cuda()
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 81 %


In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data                       # this is what you had
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')



[1,  2000] loss: 1.861
[1,  4000] loss: 1.470
[1,  6000] loss: 1.255
[1,  8000] loss: 1.135
[1, 10000] loss: 1.043
[1, 12000] loss: 0.969
[2,  2000] loss: 0.857
[2,  4000] loss: 0.827
[2,  6000] loss: 0.809
[2,  8000] loss: 0.738
[2, 10000] loss: 0.738
[2, 12000] loss: 0.703
[3,  2000] loss: 0.577
[3,  4000] loss: 0.620
[3,  6000] loss: 0.611
[3,  8000] loss: 0.583
[3, 10000] loss: 0.567
[3, 12000] loss: 0.561
[4,  2000] loss: 0.448
[4,  4000] loss: 0.443
[4,  6000] loss: 0.459
[4,  8000] loss: 0.461
[4, 10000] loss: 0.458
[4, 12000] loss: 0.468
[5,  2000] loss: 0.316
[5,  4000] loss: 0.326
[5,  6000] loss: 0.344
[5,  8000] loss: 0.347
[5, 10000] loss: 0.375
[5, 12000] loss: 0.368
[6,  2000] loss: 0.232
[6,  4000] loss: 0.255
[6,  6000] loss: 0.276
[6,  8000] loss: 0.264
[6, 10000] loss: 0.270
[6, 12000] loss: 0.269
[7,  2000] loss: 0.180
[7,  4000] loss: 0.166
[7,  6000] loss: 0.191
[7,  8000] loss: 0.189
[7, 10000] loss: 0.213
[7, 12000] loss: 0.223
[8,  2000] loss: 0.111
[8,  4000] 

In [27]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        images =images.cuda()
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %


In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=44, kernel_size=3, padding=1),
            nn.BatchNorm2d(44),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=44, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data                       # this is what you had
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')



[1,  2000] loss: 1.866
[1,  4000] loss: 1.501
[1,  6000] loss: 1.318
[1,  8000] loss: 1.161
[1, 10000] loss: 1.041
[1, 12000] loss: 0.967
[2,  2000] loss: 0.857
[2,  4000] loss: 0.825
[2,  6000] loss: 0.794
[2,  8000] loss: 0.753
[2, 10000] loss: 0.737
[2, 12000] loss: 0.716
[3,  2000] loss: 0.603
[3,  4000] loss: 0.609
[3,  6000] loss: 0.602
[3,  8000] loss: 0.588
[3, 10000] loss: 0.588
[3, 12000] loss: 0.562
[4,  2000] loss: 0.443
[4,  4000] loss: 0.462
[4,  6000] loss: 0.443
[4,  8000] loss: 0.464
[4, 10000] loss: 0.456
[4, 12000] loss: 0.454
[5,  2000] loss: 0.330
[5,  4000] loss: 0.329
[5,  6000] loss: 0.364
[5,  8000] loss: 0.343
[5, 10000] loss: 0.381
[5, 12000] loss: 0.358
[6,  2000] loss: 0.233
[6,  4000] loss: 0.249
[6,  6000] loss: 0.262
[6,  8000] loss: 0.273
[6, 10000] loss: 0.276
[6, 12000] loss: 0.287
[7,  2000] loss: 0.174
[7,  4000] loss: 0.177
[7,  6000] loss: 0.204
[7,  8000] loss: 0.193
[7, 10000] loss: 0.203
[7, 12000] loss: 0.202
[8,  2000] loss: 0.120
[8,  4000] 

In [29]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        images =images.cuda()
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 82 %
